In [1]:
import sys
import re
sys.path.append(r'D:/backup/PRO/')
from  lib.initiate_ssh_connection import SSHClient
import mysql.connector
import socket
import json
import threading
import time
import requests

In [ ]:
def check_ping(devices):
    ping_status = dict()
    for ip, tag in devices.items():
        try:
            # Create a socket object and attempt to connect to the IP address on port 80
            sock = socket.create_connection((ip, 80), timeout=2)
            sock.close()
            ping_status[ip] = 'UP'
        except (socket.timeout, socket.error):
            ping_status[ip] = 'DOWN'
    return ping_status, [i for i in ping_status.keys() if ping_status[i]=='UP']

In [ ]:
def check_ping(devices):
    status_info = {}
    ssh_list  = []
    for tag, msip in devices:
        ping_status = {}
        for ip in msip:
            try:
            # Create a socket object and attempt to connect to the IP address on port 80
                sock = socket.create_connection((ip, 80), timeout=2)
                sock.close()
                ping_status[ip] = 'UP'
            except(socket.timeout, socket.error):
                ping_status[ip] = 'DOWN'
        status_info[tag] = ping_status
    for tag,dev_list in status_info.items():
        for ip,status in dev_list.items():
            if status=='UP':
                ssh_list.append(ip)
                break
    return status_info, ssh_list
            
            

In [6]:
def check_ping(devices):
    status_info = {}
    ssh_list  = []
    for tag, msip in devices.items():
        ping_status = {}
        for ip in msip:
            try:
            # Create a socket object and attempt to connect to the IP address on port 80
                sock = socket.create_connection((ip, 80), timeout=2)
                sock.close()
                ping_status[ip] = 'UP'
            except(socket.timeout, socket.error):
                ping_status[ip] = 'DOWN'
        status_info[tag] = ping_status
    for tag,dev_list in status_info.items():
        for ip,status in dev_list.items():
            if status=='UP':
                ssh_list.append(ip)
                break
    return status_info, ssh_list

In [7]:
dev = {'SAMPLE-DUT': {'192.168.244.109', '192.168.107.109', '192.168.109.109'}, 'Sample-Dut-2': {None, '192.168.107.20'}, 'LAN-DEVICE': {'192.168.107.21', None}, 'E8K-Manu': {'192.168.107.237', None}, 'HA-DUT-1': {'192.168.107.95', None}, 'MSA-VM': {'192.168.109.137', None}, 'HA-DUT': {None, '192.168.107.96'}, 'BGP-SERVER': {'192.168.107.71', None}, 'vm': {'192.168.1.1', None}, 'TVM': {'2.2.2.2', None}}

In [9]:
l,b = check_ping(dev)

In [11]:
b

['192.168.109.109',
 '192.168.107.20',
 '192.168.107.21',
 '192.168.107.237',
 '192.168.109.137',
 '192.168.107.96']

In [15]:
ignore_interface = ['cni-podman0', 'fptun0',  'lo',  'veth', 'vneth', 'vnet','io','cni-podman0', 'eth-int', 'vethb', 'vnet', 'vnet','cni-podman1','docker0']

In [27]:
def sql_connect():
    host = "192.168.109.137"
    port = 3306
    user = "root"
    password = "password"
    database = "DEVICE_TRACKER"

    # Establish a connection to the database
    connection = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    )
    return connection
# Function for retreving data from mysql database
def get_dev_details():
    dev = {}
    connection = sql_connect()
    try:
    # Create a cursor to execute SQL queries
        cursor = connection.cursor()

        # Define the SQL query to retrieve IP and TAG from DEVICES table
        query = r"SELECT ip, tags, ip2, ip3 FROM field"

        # Execute the query
        cursor.execute(query)

        # Fetch all the rows
        rows = cursor.fetchall()

        # Print the results
        for row in rows:
            ip, tag, ip2, ip3 = row
            dev[tag] = {ip, ip2, ip3}

    except:
        pass
  #  cursor.close()
    connection.close()
    return dev
# Function for retriving username and password of spcific device
def get_pass_details(ip):
    connection = sql_connect()
    cursor = connection.cursor()
    query =  r"SELECT username, password FROM field WHERE ip = %s OR ip2 = %s OR ip3 = %s"
    cursor.execute(query, (ip,ip,ip))
    cred = cursor.fetchone()
    connection.close()
    return cred

In [25]:
def procees_interface(inf):
    final_inf = []
    try:
        inf.remove('')
    except ValueError:
        pass

    for i in inf:
        if i[-1] == ':':
            final_inf.append(i[:-1])
        else:
            final_inf.append(i)

    final_inf = [interface for interface in final_inf if not any(ignore in interface for ignore in ignore_interface)]

    return final_inf

In [22]:
b

['192.168.109.109',
 '192.168.107.20',
 '192.168.107.21',
 '192.168.107.237',
 '192.168.109.137',
 '192.168.107.96']

In [18]:
def initiate_ssh(l):
    status_check = {}
    layer_status = {}
    for ip in l:
            ifconfig_status = ''
            route_status = ''
            arp_status = ''
            status = {}
            l_status = {}
            username, password = get_pass_details(ip)
            obj = SSHClient(ip,username,password)
            obj.connect()
            try:
                interface = str(obj.run_execmcd("ifconfig  | awk '/^[a-zA-Z]/{print $1}'"))
                interface = interface.split('\n')        
                interface = procees_interface(interface)
                ifconfig_status = obj.run_execmcd('ifconfig')
                route_status = obj.run_execmcd('route -n')
                arp_status = obj.run_execmcd('arp -a')
                for i in interface:
                    link = obj.run_execmcd(f"ethtool {i} | grep 'Link detected' | awk '{{print $3}}'")
                    status[i] = link.strip()
                status_check[ip] = status
                l_status['ifconfig'] = ifconfig_status
                l_status['route'] = route_status
                l_status['arp'] = arp_status
                layer_status[ip] = l_status
                obj.close_connection()
            except:
                print("Issue facing while running ethtool") # Failed to SSH
    return json.dumps(status_check), json.dumps(layer_status)


In [19]:
initiate_ssh(b)

TypeError: cannot unpack non-iterable NoneType object

In [28]:
username, password = get_pass_details('192.168.109.109')

In [29]:
username

'root'